In [67]:
import networkx as nx
from tqdm import tqdm
from datetime import datetime

In [68]:
start = datetime.now()
file = open('ca-AstroPh.txt','r')
lines = file.readlines()

In [69]:
def dic2list(myDic):
    graph = []
    for i in myDic.keys():
        graph.append(i)
    return graph

In [70]:
nodes = {}
unique_nodes = set()
neighbor_c = {}
for l in lines:
    if '#' in str(l):
        continue
    node = l.replace('\n','').split('\t')
    n1 = int(node[0])
    n2 = int(node[1])
    nodes[(n1,n2)] = 1
    unique_nodes.add(n1)
    unique_nodes.add(n2)
    if  neighbor_c.get(n1,0) == 0:
             neighbor_c[n1] = []
    neighbor_c[n1].append(n2)
nodes_set = dic2list(nodes)

In [71]:
def my_bfs(visited, queue, graph, node):
    visited.append(node)
    queue.append(node)
    n_list = []
    while queue:
        s = queue.pop(0)
        n_list.append(s)
        # print(graph[s])
        for neighbour in graph[s]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
    return n_list

In [72]:
def iter_bfs_2(neighbor):
    visited_n = []
    queue_n = []
    n_neighbor = {}
    for a in tqdm(neighbor):
        n_neighbor[a] = my_bfs(queue_n, visited_n,neighbor_c,a)
    return n_neighbor

In [73]:
hc_bfsed =  iter_bfs_2(unique_nodes)

100%|██████████| 18772/18772 [01:20<00:00, 233.38it/s]


In [74]:
out_sorted = sorted(hc_bfsed, key=lambda k: len(hc_bfsed[k]), reverse=True).copy()

### Greedy solution

In [75]:
def hill_climbing(hc_s):
    unique_nodes_h = unique_nodes.copy()
    neigh = set()
    for s in hc_s:
        unique_nodes_h.remove(s)
        neigh.add(frozenset(hc_bfsed[s]))
    while  len(hc_s) < 10 :
        temp_dic = {}
        for n in unique_nodes_h:
            temp_dic[n] = neigh.union(hc_bfsed[n])
        selected = sorted(temp_dic, key=lambda k: len(temp_dic[k]), reverse=True)[0]
        neigh.add(frozenset(temp_dic[selected]))
        hc_s.append(selected)
        unique_nodes_h.remove(selected)
    return hc_s
S = []
S.append(out_sorted[0])

hc = hill_climbing(S)
hc

[3, 99974, 38863, 67720, 65252, 33128, 358, 131978, 100471, 885]

###  Celf with lazy Hill climbing (Marginal gain version)

In [76]:
lhc_bfs = hc_bfsed.copy()

In [77]:
def lazy_hill_climbing():
    sl = []
    neighbor_sorted_l = out_sorted.copy()
    s1 = neighbor_sorted_l[0]
    sl.append(s1)
    sl_outs = lhc_bfs[s1].copy()

    neighbor_sorted_l.remove(s1)
    for i, n in enumerate(neighbor_sorted_l):
        if len(sl) > 9:
            break
        left = n
        right = neighbor_sorted_l[i + 1]

        mg_l = [a for a in lhc_bfs[left] if a not in sl_outs]

        l = len(mg_l)
        r = len(lhc_bfs[right])

        if l >= r:
            sl.append(n)
            sl_outs.append(lhc_bfs[n])
        else:
            lhc_bfs[n] = mg_l.copy()
            neighbor_sorted_l = sorted(lhc_bfs, key=lambda k: len(lhc_bfs[k]), reverse=True).copy()
    return sl
lhc = lazy_hill_climbing()
lhc

[3, 99974, 38863, 65252, 67720, 33128, 358, 131978, 100471, 885]

### Celf with lazy Hill climbing (benefit cost version)

In [78]:
cost = {v: 1 for v in unique_nodes}
benefit_cost_ratio = {v: len(hc_bfsed[v]) / cst for v, cst in cost.items()}

In [79]:
cost = {v:1 for v in unique_nodes}
mg_bc = {v: len(hc_bfsed[v]) / c for v,c in cost.items()}
mg_sorted = sorted(mg_bc,key=lambda k:mg_bc[k],reverse=True)

In [80]:
def mg_calc(ne,node):
    mg_neighbor = neighbor_c.copy()
    for o in ne:
        del mg_neighbor[o]
    return len(my_bfs([],[],mg_neighbor,node))/1

In [81]:
def lhc_celf():
    sl = []
    mg_sorted_c = mg_sorted.copy()
    mg_bc_c = mg_bc.copy()
    s1 = mg_sorted_c[0]
    sl.append(s1)
    sl_outs = hc_bfsed[s1]
    for i , m in enumerate(mg_sorted_c):
        if len(sl) > 9:
            break
        left = m
        right = mg_sorted_c[i+1]
        l=mg_bc_c[left]
        r=mg_bc_c[right]
        if l >= r:
            if i != 0:
                sl.append(m)
            sl_outs.append(hc_bfsed[m])
            mg_bc_c[i+1] = mg_calc(sl,mg_sorted_c[i+1])
            mg_sorted_c = sorted(mg_bc,key=lambda k:mg_bc[k],reverse=True)
        else:
            pass
    return sl

lhc_c = lhc_celf()

### Verify Final Answers with NetworkX library

In [82]:
print("-----------\nhill climbing")
for i in hc:
    print(i,len(list(nx.bfs_tree(nx.DiGraph(nodes_set),i  ))))
print("-----------\nLazy hill climbing")
for i in lhc:
    print(i,len(list(nx.bfs_tree(nx.DiGraph(nodes_set),i  ))))
print("-----------\nhill climbing benefit cost")
for i in lhc_c:
    print(i,len(list(nx.bfs_tree(nx.DiGraph(nodes_set),i  ))))

-----------
hill climbing
3 17903
99974 18
38863 12
67720 10
65252 10
33128 9
358 8
131978 8
100471 8
885 7
-----------
Lazy hill climbing
3 17903
99974 18
38863 12
65252 10
67720 10
33128 9
358 8
131978 8
100471 8
885 7
-----------
hill climbing benefit cost
3 17903
99974 18
38863 12
65252 10
67720 10
33128 9
358 8
131978 8
100471 8
885 7
